# Process and calculate distance of deforestation pixel to road  and to river

In [1]:
import rasterio
import geopandas as gpd
from scipy.ndimage import distance_transform_edt
from concurrent.futures import ThreadPoolExecutor
from process_functions  import  process_columns, vector_to_raster, calculate_distance_rasters
from distance_calculation import process_year
from shapely.geometry import Polygon, MultiPolygon
from constants import SHAPEFILE_PATH


In [2]:
# Read the river_shapefile using GeoPandas
river_gdf = gpd.read_file("/Users/romero61/../../capstone/pyforest/data/roads_hydrology/ly_hid2.shp")
# Read the river_shapefile using GeoPandas
road_gdf = gpd.read_file("/Users/romero61/../../capstone/pyforest/data/roads_hydrology/ly_via2.shp")



In [3]:
river_gdf.crs

<Projected CRS: PROJCS["WGS_1984_UTM_Zone_21S",GEOGCS["WGS 84",DAT ...>
Name: WGS_1984_UTM_Zone_21S
Axis Info [cartesian]:
- [east]: Easting (Meter)
- [north]: Northing (Meter)
Area of Use:
- undefined
Coordinate Operation:
- name: UTM zone 21S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

# Reprojections

In [5]:
study_boundary_gdf = gpd.read_file(SHAPEFILE_PATH)

# Reproject the study boundary GeoDataFrame to 4326
study_boundary_gdf_reprojected = study_boundary_gdf.to_crs("EPSG:4326")
# Get the geometry column from the reprojected GeoDataFrame
study_boundary_geom_reprojected = study_boundary_gdf_reprojected.geometry



In [8]:
study_boundary_gdf_reprojected.crs


<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [12]:
# Get the total bounds of the river_shapefile
minx, miny, maxx, maxy = study_boundary_gdf_reprojected.total_bounds

# Create a Shapely Polygon from the bounds
bounds_polygon = Polygon([(minx, miny), (minx, maxy), (maxx, maxy), (maxx, miny)])

# Create a GeoDataFrame from the Polygon and name it study_bounds
study_bounds = gpd.GeoDataFrame(geometry=[bounds_polygon], crs=study_boundary_gdf_reprojected.crs)

In [13]:
study_bounds.total_bounds

array([-62.64185791, -25.35415853, -57.14930819, -19.28764722])

In [20]:
# Skeleton of process_columns inputs
# Function requires absolute path couldn't get it to take gpd
#input_vector = "/Users/romero61/../../capstone/pyforest/data/roads_hydrology/ly_hid2.shp"
input_vector = "/Users/romero61/../../capstone/pyforest/data/roads_hydrology/ly_via2.shp"



output_dir = '/Users/romero61/github/PYFOREST-ML/src/data_loading/output_rasters'
#study_area_bounds = (minx, miny, maxx, maxy)  # Replace with the bounds of your study area
#columns = ['column1', 'column2']  # Replace with the desired columns or None to use all columns

#process_columns(input_vector, output_dir, study_area_bounds, columns=columns)

In [21]:
process_columns(input_vector, output_dir, study_bounds.total_bounds, single_value=1)


Entering vector_to_raster
Writing to /Users/romero61/github/PYFOREST-ML/src/data_loading/output_rasters/value_raster.tif


In [22]:
import os

output_file_path = f"{output_dir}/value_raster.tif"  # Replace 'column_name' with the appropriate column name
if os.path.exists(output_file_path):
    print(f"Output file exists: {output_file_path}")
else:
    print(f"Output file not found: {output_file_path}")


Output file exists: /Users/romero61/github/PYFOREST-ML/src/data_loading/output_rasters/value_raster.tif


# MANUAL RENAME REQUIRED

In [23]:
deforestation_years = [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

num_cores = 19
with ThreadPoolExecutor(max_workers=num_cores) as executor:
    executor.map(process_year, deforestation_years)


/Users/romero61/../../capstone/pyforest/ml_data/deforestation_by_year/deforestation_11.tif
output_rasters/deforestation_river_distance_11.tif
output_rasters/deforestation_road_distance_11.tif
Processing deforestation data for year 2011
/Users/romero61/../../capstone/pyforest/ml_data/deforestation_by_year/deforestation_12.tif
output_rasters/deforestation_river_distance_12.tif
output_rasters/deforestation_road_distance_12.tif
Processing deforestation data for year 2012
/Users/romero61/../../capstone/pyforest/ml_data/deforestation_by_year/deforestation_13.tif
output_rasters/deforestation_river_distance_13.tif
output_rasters/deforestation_road_distance_13.tif
Processing deforestation data for year 2013
/Users/romero61/../../capstone/pyforest/ml_data/deforestation_by_year/deforestation_14.tif
output_rasters/deforestation_river_distance_14.tif
output_rasters/deforestation_road_distance_14.tif
Processing deforestation data for year 2014
/Users/romero61/../../capstone/pyforest/ml_data/deforest

In [25]:
# using a loop instead of parallelized code
deforestation_years = [11]

for year in deforestation_years:
    deforestation_raster = f"/Users/romero61/../../capstone/pyforest/ml_data/deforestation_by_year/deforestation_{year}.tif"
    river_distance_output = f"output_rasters/deforestation_river_distance_{year}.tif"
    road_distance_output = f"output_rasters/deforestation_road_distance_{year}.tif"

    print(f"Processing deforestation data for year {year + 2000}")
    calculate_distance_rasters(
    deforestation_raster,
    "/Users/romero61/github/PYFOREST-ML/src/data_loading/output_rasters/rivers_value_raster.tif",
    "/Users/romero61/github/PYFOREST-ML/src/data_loading/output_rasters/road_value_raster.tif",
    river_distance_output,
    road_distance_output
)

Processing deforestation data for year 2011
Entering calculate_distance_rasters


In [ ]:
# Get the band names from the clipped raster
band_names = clipped_hansen_dataset.bandNames().getInfo()

# Print the band names
print("Band names:")
for i, band in enumerate(band_names, 1):
    print(f"{i}. {band}")


C:/Users/bsf31/Desktop/capstone/deforestation_river_distance_11.tif, band 1: IReadBlock failed at X offset 0, Y offset 22302: TIFFReadEncodedStrip() failed.
C:/Users/bsf31/Desktop/capstone/deforestation_river_distance_11.tif, band 1: IReadBlock failed at X offset 0, Y offset 7212: TIFFReadEncodedStrip() failed.
C:/Users/bsf31/Desktop/capstone/deforestation_river_distance_11.tif, band 1: IReadBlock failed at X offset 0, Y offset 4966: TIFFReadEncodedStrip() failed.
C:/Users/bsf31/Desktop/capstone/deforestation_river_distance_11.tif, band 1: IReadBlock failed at X offset 0, Y offset 4826: TIFFReadEncodedStrip() failed.
C:/Users/bsf31/Desktop/capstone/deforestation_river_distance_11.tif, band 1: IReadBlock failed at X offset 0, Y offset 4837: TIFFReadEncodedStrip() failed.
C:/Users/bsf31/Desktop/capstone/deforestation_river_distance_11.tif, band 1: IReadBlock failed at X offset 0, Y offset 4834: TIFFReadEncodedStrip() failed.